In [7]:
import re
import json
from xml.dom import minidom

import xml.etree.ElementTree as ET
from urdf2mjcf.model import DefaultJointMetadata

ROBOT_CLASS = "robot"

default_metadata = "/home/tatp/ws/GAL/urdf2mjcf/tests/foxtrot/default.json"
with open(default_metadata, "r") as f:
    default_metadata = json.load(f)
    for key, value in default_metadata.items():
        default_metadata[key] = DefaultJointMetadata.from_dict(value)

In [ ]:
default_metadata['leg_joint'].joint

In [11]:
default = ET.Element("default")
robot_default = ET.SubElement(default, "default", attrib={"class": ROBOT_CLASS})

for class_name, class_metadata in default_metadata.items():
    sub_default = ET.SubElement(robot_default, "default", attrib={"class": str(class_name)})
    joint_attrib = {}
    c_joint = class_metadata.joint
    if c_joint.armature is not None:
        joint_attrib["armature"] = str(c_joint.armature)
    if c_joint.stiffness is not None:
        joint_attrib["stiffness"] = str(c_joint.stiffness)
    if c_joint.damping is not None:
        joint_attrib["damping"] = str(c_joint.damping)
    if c_joint.frictionloss is not None:
        joint_attrib["frictionloss"] = str(c_joint.frictionloss)
    ET.SubElement(sub_default, "joint", attrib=joint_attrib)

    c_actuator = class_metadata.actuator
    if c_actuator.actuator_type is None:
        continue

    actuator_attrib = {}
    if c_actuator.kp is not None:
        actuator_attrib["kp"] = str(c_actuator.kp)
    if c_actuator.kv is not None:
        actuator_attrib["kv"] = str(c_actuator.kv)
    if c_actuator.forcerange is not None:
        actuator_attrib["forcerange"] = f"{c_actuator.forcerange[0]} {c_actuator.forcerange[1]}"

    ET.SubElement(sub_default, c_actuator.actuator_type, attrib=actuator_attrib)


In [ ]:
xmlstr = minidom.parseString(ET.tostring(default)).toprettyxml(indent="  ")
xmlstr = re.sub(r"\n\s*\n", "\n", xmlstr)

# Add newlines between second-level nodes
root = ET.fromstring(xmlstr)
for child in root[:-1]:
    child.tail = "\n\n  "
xmlstr = ET.tostring(root, encoding="unicode")
print(xmlstr)